### previously - workbook3 ;  in continuation - take preprocessSTG8_d2.csv
### Bert Prep
#### ---- set D2 - Lifestyle, Health & Fitness, Tools

In [1]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('preprocessSTG8_d2.csv')
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Calendar,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length
0,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,7662,False,Ampere Battery Info helps you detect if you ar...,https://dev4phones.wordpress.com/licencia-de-uso/,0,0,...,0,0,0,0,0,0,0,975,ampere battery info helps you detect if you ar...,890
1,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,702,False,he Every Day Calendar is a mobile application ...,https://www.freeprivacypolicy.com/privacy/view...,0,1,...,1,1,0,1,1,0,0,336,he every day calendar is a mobile application ...,336


In [3]:
df.shape

(217838, 22)

In [4]:
df.dropna(subset=['Clean_Description'],inplace=True)

In [5]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [7]:
!pip install transformers

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [9]:
import numpy as np

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# Record the length of each sequence (after truncating to 512)
lengths = []

print('Tokenizing descriptions...')

# For every sentence...
for sent in df.Clean_Description:

    # Report progress
    if ((len(input_ids) % 20000) == 0):
        print('   Read {:,} descriptions.'.format(len(input_ids)))

    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

    # Record the truncated length.
    lengths.append(len(encoded_sent))

print('DONE. ')
print('{:>10,} comments'.format(len(input_ids)))

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


Tokenizing descriptions...
   Read 0 descriptions.
   Read 20,000 descriptions.
   Read 40,000 descriptions.
   Read 60,000 descriptions.
   Read 80,000 descriptions.
   Read 100,000 descriptions.
   Read 120,000 descriptions.
   Read 140,000 descriptions.
   Read 160,000 descriptions.
   Read 180,000 descriptions.
   Read 200,000 descriptions.
DONE. 
   217,795 comments


In [10]:
print(lengths)
print(len(lengths))
print(df.shape)

[173, 72, 183, 60, 255, 76, 64, 40, 97, 73, 251, 230, 87, 308, 522, 86, 59, 90, 411, 28, 209, 520, 33, 13, 55, 167, 451, 128, 178, 67, 17, 17, 162, 89, 158, 42, 494, 55, 157, 283, 85, 150, 24, 46, 45, 226, 350, 641, 134, 98, 527, 88, 69, 45, 177, 102, 49, 346, 107, 269, 274, 53, 206, 201, 214, 136, 494, 70, 77, 33, 122, 23, 138, 110, 373, 44, 355, 255, 88, 64, 73, 559, 102, 776, 240, 390, 104, 35, 18, 51, 31, 109, 12, 617, 168, 92, 30, 44, 673, 108, 188, 131, 209, 55, 67, 88, 493, 176, 95, 227, 76, 171, 281, 320, 238, 14, 25, 38, 51, 54, 30, 267, 858, 337, 354, 65, 77, 77, 104, 46, 224, 201, 120, 52, 28, 333, 72, 457, 294, 216, 98, 95, 81, 280, 213, 518, 140, 676, 34, 20, 85, 19, 92, 300, 45, 458, 172, 309, 142, 91, 163, 92, 128, 30, 22, 22, 207, 92, 250, 33, 194, 198, 209, 209, 118, 95, 397, 127, 96, 660, 54, 55, 103, 124, 10, 31, 34, 116, 73, 55, 33, 53, 375, 386, 41, 80, 131, 39, 161, 567, 702, 357, 307, 255, 269, 649, 80, 63, 815, 35, 61, 27, 35, 16, 41, 249, 103, 221, 370, 45, 360

In [11]:
df['token_length'] = lengths
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,7662,False,Ampere Battery Info helps you detect if you ar...,https://dev4phones.wordpress.com/licencia-de-uso/,0,0,...,0,0,0,0,0,0,975,ampere battery info helps you detect if you ar...,890,173
1,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,702,False,he Every Day Calendar is a mobile application ...,https://www.freeprivacypolicy.com/privacy/view...,0,1,...,1,0,1,1,0,0,336,he every day calendar is a mobile application ...,336,72


In [12]:
print(sum(df['token_length'] < 25))

13212


In [13]:
print(sum(df['token_length'] >= 510))

15696


In [14]:
print('    Min length: {:,} tokens'.format(min(lengths)))
print('    Min length: {:,} tokens'.format(max(lengths)))
print(' Median length: {:,} tokens'.format(np.median(lengths)))

    Min length: 3 tokens
    Min length: 1,571 tokens
 Median length: 121.0 tokens


In [15]:
df.head(3)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,7662,False,Ampere Battery Info helps you detect if you ar...,https://dev4phones.wordpress.com/licencia-de-uso/,0,0,...,0,0,0,0,0,0,975,ampere battery info helps you detect if you ar...,890,173
1,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,702,False,he Every Day Calendar is a mobile application ...,https://www.freeprivacypolicy.com/privacy/view...,0,1,...,1,0,1,1,0,0,336,he every day calendar is a mobile application ...,336,72
2,All in one shopping app,com.tkapplications.allinoneshoppingapp,Tools,5.0,62,False,Here is an app where you will find the desired...,NaN,0,0,...,0,0,0,0,0,0,781,here is an app where you will find the desired...,735,183


In [ ]:
## save the df dataset file - 

In [16]:
df.to_csv("preprocessSTG9_d2.csv",index=False)